In [ ]:
SINGLE_CSV = "/content/IMDB_Dataset_clean.csv"
OUT_DIR    = "/content"
SEED       = 42
TRAIN_R, VAL_R, TEST_R = 0.80, 0.10, 0.10

import os, json, numpy as np, pandas as pd
assert os.path.exists(SINGLE_CSV), f"File tidak ditemukan: {SINGLE_CSV}"

df = pd.read_csv(SINGLE_CSV, sep=None, engine="python", encoding="utf-8-sig")
print("Loaded:", len(df), "rows")
print("Columns:", list(df.columns))

label_candidates = ["label", "sentiment", "target", "class", "category", "polarity"]
lower_map = {c.lower(): c for c in df.columns}
label_col = next((lower_map[c] for c in label_candidates if c in lower_map), None)
stratified = label_col is not None and df[label_col].nunique(dropna=True) > 1
print("Stratified by:", label_col if stratified else "None (random split)")

rng = np.random.RandomState(SEED)

def split_indices_random(n, tr, vr):
    perm = np.arange(n); rng.shuffle(perm)
    n_tr = int(round(n*tr)); n_v = int(round(n*vr))
    if n_tr + n_v > n: n_v = max(0, n - n_tr)
    return perm[:n_tr], perm[n_tr:n_tr+n_v], perm[n_tr+n_v:]

def split_indices_stratified(df, label_col, tr, vr):
    train_idx, val_idx, test_idx = [], [], []
    for _, sub in df.groupby(label_col, dropna=False):
        idx = sub.index.to_numpy()
        rng.shuffle(idx)
        n = len(idx)
        n_tr = int(round(n*tr)); n_v = int(round(n*vr))
        if n_tr + n_v > n: n_v = max(0, n - n_tr)
        train_idx.append(idx[:n_tr])
        val_idx.append(idx[n_tr:n_tr+n_v])
        test_idx.append(idx[n_tr+n_v:])
    return (np.concatenate(train_idx), np.concatenate(val_idx), np.concatenate(test_idx))

if stratified:
    tr_idx, va_idx, te_idx = split_indices_stratified(df, label_col, TRAIN_R, VAL_R)
else:
    tr_idx, va_idx, te_idx = split_indices_random(len(df), TRAIN_R, VAL_R)


os.makedirs(os.path.join(OUT_DIR, "splits"), exist_ok=True)
with open(os.path.join(OUT_DIR, "splits", "split_indices.json"), "w", encoding="utf-8") as f:
    json.dump({"train": tr_idx.tolist(), "val": va_idx.tolist(), "test": te_idx.tolist()}, f, ensure_ascii=False, indent=2)

# tulis CSV
df_train = df.loc[tr_idx].sample(frac=1.0, random_state=SEED).reset_index(drop=True)
df_val   = df.loc[va_idx].sample(frac=1.0, random_state=SEED).reset_index(drop=True)
df_test  = df.loc[te_idx].sample(frac=1.0, random_state=SEED).reset_index(drop=True)

train_path = os.path.join(OUT_DIR, "train.csv")
val_path   = os.path.join(OUT_DIR, "val.csv")
test_path  = os.path.join(OUT_DIR, "test.csv")
df_train.to_csv(train_path, index=False, encoding="utf-8")
df_val.to_csv(val_path, index=False, encoding="utf-8")
df_test.to_csv(test_path, index=False, encoding="utf-8")

print("\nSaved:")
print(" -", train_path, len(df_train))
print(" -", val_path,   len(df_val))
print(" -", test_path,  len(df_test))
if stratified:
    print("\nLabel dist:")
    print("train:", df_train[label_col].value_counts(dropna=False).to_dict())
    print("val  :", df_val[label_col].value_counts(dropna=False).to_dict())
    print("test :", df_test[label_col].value_counts(dropna=False).to_dict())


Loaded: 50000 rows
Columns: ['review', 'sentiment']
Stratified by: sentiment

Saved:
 - /content/train.csv 40000
 - /content/val.csv 5000
 - /content/test.csv 5000

Label dist:
train: {'positive': 20000, 'negative': 20000}
val  : {'negative': 2500, 'positive': 2500}
test : {'negative': 2500, 'positive': 2500}


In [ ]:
TRAIN_CSV  = "/content/train.csv"
VAL_CSV    = "/content/val.csv"
TEST_CSV   = "/content/test.csv"

import csv, re, os, time, random
from dataclasses import dataclass
from typing import List, Tuple, Dict
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader

@dataclass
class Config:
    train: str = TRAIN_CSV
    val: str   = VAL_CSV
    test: str  = TEST_CSV
    max_len: int=128; batch_size:int=64; epochs:int=10; lr:float=1e-3
    emb_dim:int=128; hidden:int=128; num_layers:int=1; dropout:float=0.2
    min_freq:int=5; seed:int=42; grad_clip:float=1.0
cfg = Config()

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def set_seed(s:int):
    random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    try: import numpy as np; np.random.seed(s)
    except: pass
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False
set_seed(cfg.seed)

def detect_delimiter(path):
    with open(path,"r",encoding="utf-8-sig",newline="") as f: first=f.readline()
    return ";" if first.count(";")>first.count(",") else ","
def resolve_text_label_idx(headers):
    ti, li = 0, 1 if len(headers)>1 else (0,0)
    lower=[h.strip().lstrip("\ufeff").lower() for h in headers]
    for i,n in enumerate(lower):
        if n in ("text","review","sentence"): ti=i
        if n in ("label","sentiment","target"): li=i
    return ti, li
def read_csv_flex(path):
    delim=detect_delimiter(path); out=[]
    with open(path,"r",encoding="utf-8-sig",newline="") as f:
        reader=csv.reader(f,delimiter=delim); headers=next(reader,None)
        assert headers, f"Empty header: {path}"
        ti,li=resolve_text_label_idx(headers)
        for rec in reader:
            if not rec or len(rec)==1: continue
            if li<len(rec) and ti<len(rec): txt,lab=rec[ti],rec[li]
            else: lab=rec[-1]; txt=",".join(rec[:-1])
            txt,lab=(txt or "").strip(),(lab or "").strip()
            if txt and lab: out.append((txt,lab))
    return out

TOKEN_RE=re.compile(r"[A-Za-z0-9]+")
class Vocab:
    def __init__(self): self.itos=["<pad>","<unk>"]; self.stoi={w:i for i,w in enumerate(self.itos)}; self.unk_id=1
    def add(self,t):
        if t in self.stoi: return self.stoi[t]
        self.stoi[t]=len(self.itos); self.itos.append(t); return self.stoi[t]
    def get(self,t): return self.stoi.get(t,self.unk_id)
    def __len__(self): return len(self.itos)
def tok(s): return TOKEN_RE.findall(s.lower())
def build_vocab(rows,min_freq):
    from collections import Counter
    cnt=Counter(); [cnt.update(tok(x)) for x,_ in rows]
    v=Vocab()
    for t,c in sorted(cnt.items(), key=lambda x:x[1]):
        if c>=min_freq: v.add(t)
    return v
def build_labels(rows):
    labs=sorted(set(l for _,l in rows)); return {lab:i for i,lab in enumerate(labs)}

class TextDataset(Dataset):
    def __init__(self, rows, vocab, labmap, max_len):
        self.data=[]
        for txt,lab in rows:
            ids=[vocab.get(t) for t in tok(txt)]
            if len(ids)>max_len: ids=ids[:max_len]
            self.data.append((ids,len(ids),labmap[lab]))
        self.max_len=max_len
    def __len__(self): return len(self.data)
    def __getitem__(self,i): return self.data[i]

def collate_pad(batch,pad_id=0,max_len=None):
    lens=[b[1] for b in batch]; ys=torch.tensor([b[2] for b in batch],dtype=torch.long)
    if max_len is None: max_len=max(lens) if lens else 1
    xs=[(ids+[pad_id]*(max_len-l)) for ids,l,_ in batch]
    return torch.tensor(xs,dtype=torch.long), torch.tensor(lens), ys

class BiLstmClf(nn.Module):
    def __init__(self,vocab_size,emb,hidden,layers,num_classes,drop):
        super().__init__()
        self.embed=nn.Embedding(vocab_size,emb,padding_idx=0)
        self.lstm=nn.LSTM(emb,hidden,num_layers=layers,batch_first=True,bidirectional=True)
        self.fc=nn.Linear(hidden*2,num_classes); self.drop=nn.Dropout(drop)
    def forward(self,x,lens,train=True):
        emb=self.embed(x);
        if train: emb=self.drop(emb)
        out,_=self.lstm(emb) # (B,T,2H)
        B,T,H2=out.shape
        rng=torch.arange(T,device=out.device).unsqueeze(0)
        mask=(rng<lens.unsqueeze(1)).float()
        pooled=(out*mask.unsqueeze(-1)).sum(dim=1)/lens.clamp(min=1).unsqueeze(1).float()
        if train: pooled=self.drop(pooled)
        return self.fc(pooled)

@torch.no_grad()
def evaluate(model,loader,num_classes,device):
    model.eval(); ce=nn.CrossEntropyLoss()
    total_loss,total=0.0,0; conf=torch.zeros(num_classes,num_classes,dtype=torch.long,device=device)
    for x,lens,y in loader:
        x,lens,y=x.to(device),lens.to(device),y.to(device)
        logits=model(x,lens,train=False); loss=ce(logits,y)
        total_loss+=loss.item()*y.size(0); total+=y.size(0)
        pred=logits.argmax(dim=-1); idx=y*num_classes+pred
        binc=torch.bincount(idx,minlength=num_classes*num_classes)
        conf+=binc.view(num_classes,num_classes)
    tp=conf.diag().float(); fp=conf.sum(0).float()-tp; fn=conf.sum(1).float()-tp
    prec=torch.where(tp+fp>0,tp/(tp+fp),torch.zeros_like(tp))
    rec =torch.where(tp+fn>0,tp/(tp+fn),torch.zeros_like(tp))
    f1  =torch.where(prec+rec>0,2*prec*rec/(prec+rec),torch.zeros_like(tp))
    acc=(tp.sum()/conf.sum().clamp_min(1)).item()
    return (total_loss/max(total,1), acc, prec.mean().item(), rec.mean().item(), f1.mean().item())

def main():
    print("Device:", DEVICE, "| Mode: BIDIRECTIONAL")
    t0=time.time()
    tr=read_csv_flex(cfg.train); va=read_csv_flex(cfg.val); te=read_csv_flex(cfg.test)
    assert tr and va and te, "Pastikan train/val/test CSV ada & non-empty."
    vocab=build_vocab(tr,cfg.min_freq); labmap=build_labels(tr); C=len(labmap)
    print(f"Vocab={len(vocab)} | Labels={labmap}")
    ds_tr=TextDataset(tr,vocab,labmap,cfg.max_len); ds_va=TextDataset(va,vocab,labmap,cfg.max_len); ds_te=TextDataset(te,vocab,labmap,cfg.max_len)
    g=torch.Generator(); g.manual_seed(cfg.seed)
    dl_tr=DataLoader(ds_tr,batch_size=cfg.batch_size,shuffle=True,collate_fn=lambda b:collate_pad(b,0,cfg.max_len),generator=g)
    mkfix=lambda ds: DataLoader(ds,batch_size=cfg.batch_size,shuffle=False,collate_fn=lambda b:collate_pad(b,0,cfg.max_len))
    dl_va,dl_te=mkfix(ds_va),mkfix(ds_te)
    model=BiLstmClf(len(vocab),cfg.emb_dim,cfg.hidden,cfg.num_layers,C,cfg.dropout).to(DEVICE)
    opt=torch.optim.Adam(model.parameters(),lr=cfg.lr); ce=nn.CrossEntropyLoss()
    for ep in range(1,cfg.epochs+1):
        t_ep=time.time(); t_tr=time.time()
        model.train(); sum_loss=sum_acc=cnt=0
        for x,lens,y in dl_tr:
            x,lens,y=x.to(DEVICE),lens.to(DEVICE),y.to(DEVICE)
            logits=model(x,lens,train=True); loss=ce(logits,y)
            opt.zero_grad(set_to_none=True); loss.backward()
            if cfg.grad_clip: nn.utils.clip_grad_norm_(model.parameters(),cfg.grad_clip)
            opt.step()
            with torch.no_grad(): acc=(logits.argmax(-1)==y).float().mean().item()
            b=y.size(0); sum_loss+=loss.item()*b; sum_acc+=acc*b; cnt+=b
        tr_s=time.time()-t_tr
        v_loss,v_acc,p,r,f1=evaluate(model,dl_va,C,DEVICE); val_s=time.time()-t_ep-tr_s
        print(f"Epoch {ep:02d} | train loss {sum_loss/max(cnt,1):.4f} acc {sum_acc/max(cnt,1):.3f} | val loss {v_loss:.4f} acc {v_acc:.3f} P {p:.3f} R {r:.3f} F1 {f1:.3f} | time train {tr_s:.1f}s val {val_s:.1f}s total {time.time()-t_ep:.1f}s")
    t_test=time.time(); tl,ta,tp,trr,tf1=evaluate(model,dl_te,C,DEVICE)
    print(f"TEST | loss {tl:.4f} | acc {ta:.3f} | P {tp:.3f} R {trr:.3f} F1 {tf1:.3f} | time {time.time()-t_test:.1f}s")
    print(f"TOTAL {time.time()-t0:.1f}s")

if __name__=="__main__": main()


Device: cpu | Mode: BIDIRECTIONAL
Vocab=36148 | Labels={'negative': 0, 'positive': 1}
Epoch 01 | train loss 0.4964 acc 0.750 | val loss 0.3699 acc 0.832 P 0.834 R 0.832 F1 0.832 | time train 193.7s val 6.8s total 200.5s
Epoch 02 | train loss 0.3362 acc 0.852 | val loss 0.3373 acc 0.854 P 0.857 R 0.854 F1 0.854 | time train 190.8s val 6.4s total 197.2s
Epoch 03 | train loss 0.2711 acc 0.887 | val loss 0.3230 acc 0.866 P 0.866 R 0.866 F1 0.866 | time train 189.4s val 6.7s total 196.1s
Epoch 04 | train loss 0.2217 acc 0.910 | val loss 0.3225 acc 0.870 P 0.870 R 0.870 F1 0.870 | time train 189.3s val 6.4s total 195.7s
Epoch 05 | train loss 0.1766 acc 0.930 | val loss 0.3428 acc 0.865 P 0.866 R 0.865 F1 0.865 | time train 191.4s val 6.7s total 198.1s
Epoch 06 | train loss 0.1364 acc 0.948 | val loss 0.4400 acc 0.856 P 0.858 R 0.856 F1 0.856 | time train 190.2s val 6.4s total 196.6s
Epoch 07 | train loss 0.1079 acc 0.959 | val loss 0.4443 acc 0.861 P 0.862 R 0.861 F1 0.861 | time train 189.1

In [ ]:
TRAIN_CSV  = "/content/train.csv"
VAL_CSV    = "/content/val.csv"
TEST_CSV   = "/content/test.csv"

import csv, re, os, time, random
from dataclasses import dataclass
from typing import List, Tuple, Dict
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader

@dataclass
class Config:
    train: str = TRAIN_CSV
    val: str   = VAL_CSV
    test: str  = TEST_CSV
    max_len: int=128; batch_size:int=64; epochs:int=10; lr:float=1e-3
    emb_dim:int=128; hidden:int=128; num_layers:int=1; dropout:float=0.2
    min_freq:int=5; seed:int=42; grad_clip:float=1.0
cfg = Config()

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def set_seed(s:int):
    random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    try: import numpy as np; np.random.seed(s)
    except: pass
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False
set_seed(cfg.seed)

def detect_delimiter(path):
    with open(path,"r",encoding="utf-8-sig",newline="") as f: first=f.readline()
    return ";" if first.count(";")>first.count(",") else ","

def resolve_text_label_idx(headers):
    ti, li = 0, 1 if len(headers)>1 else (0,0)
    lower=[h.strip().lstrip("\ufeff").lower() for h in headers]
    for i,n in enumerate(lower):
        if n in ("text","review","sentence"): ti=i
        if n in ("label","sentiment","target"): li=i
    return ti, li

def read_csv_flex(path):
    delim=detect_delimiter(path); out=[]
    with open(path,"r",encoding="utf-8-sig",newline="") as f:
        reader=csv.reader(f,delimiter=delim); headers=next(reader,None)
        assert headers, f"Empty header: {path}"
        ti,li=resolve_text_label_idx(headers)
        for rec in reader:
            if not rec or len(rec)==1: continue
            if li<len(rec) and ti<len(rec): txt,lab=rec[ti],rec[li]
            else: lab=rec[-1]; txt=",".join(rec[:-1])
            txt,lab=(txt or "").strip(),(lab or "").strip()
            if txt and lab: out.append((txt,lab))
    return out

TOKEN_RE=re.compile(r"[A-Za-z0-9]+")
class Vocab:
    def __init__(self): self.itos=["<pad>","<unk>"]; self.stoi={w:i for i,w in enumerate(self.itos)}; self.unk_id=1
    def add(self,t):
        if t in self.stoi: return self.stoi[t]
        self.stoi[t]=len(self.itos); self.itos.append(t); return self.stoi[t]
    def get(self,t): return self.stoi.get(t,self.unk_id)
    def __len__(self): return len(self.itos)
def tok(s): return TOKEN_RE.findall(s.lower())
def build_vocab(rows,min_freq):
    from collections import Counter
    cnt=Counter(); [cnt.update(tok(x)) for x,_ in rows]
    v=Vocab()
    for t,c in sorted(cnt.items(), key=lambda x:x[1]):
        if c>=min_freq: v.add(t)
    return v
def build_labels(rows):
    labs=sorted(set(l for _,l in rows)); return {lab:i for i,lab in enumerate(labs)}

class TextDataset(Dataset):
    def __init__(self, rows, vocab, labmap, max_len):
        self.data=[]
        for txt,lab in rows:
            ids=[vocab.get(t) for t in tok(txt)]
            if len(ids)>max_len: ids=ids[:max_len]
            self.data.append((ids,len(ids),labmap[lab]))
        self.max_len=max_len
    def __len__(self): return len(self.data)
    def __getitem__(self,i): return self.data[i]

def collate_pad(batch,pad_id=0,max_len=None):
    lens=[b[1] for b in batch]; ys=torch.tensor([b[2] for b in batch],dtype=torch.long)
    if max_len is None: max_len=max(lens) if lens else 1
    xs=[(ids+[pad_id]*(max_len-l)) for ids,l,_ in batch]
    return torch.tensor(xs,dtype=torch.long), torch.tensor(lens), ys

class UniLstmClf(nn.Module):
    def __init__(self,vocab_size,emb,hidden,layers,num_classes,drop):
        super().__init__()
        self.embed=nn.Embedding(vocab_size,emb,padding_idx=0)
        self.lstm=nn.LSTM(emb,hidden,num_layers=layers,batch_first=True,bidirectional=False)
        self.fc=nn.Linear(hidden,num_classes); self.drop=nn.Dropout(drop)
    def forward(self,x,lens,train=True):
        emb=self.embed(x);
        if train: emb=self.drop(emb)
        out,_=self.lstm(emb) # (B,T,H)
        B,T,H=out.shape
        rng=torch.arange(T,device=out.device).unsqueeze(0)
        mask=(rng<lens.unsqueeze(1)).float()
        pooled=(out*mask.unsqueeze(-1)).sum(dim=1)/lens.clamp(min=1).unsqueeze(1).float()
        if train: pooled=self.drop(pooled)
        return self.fc(pooled)

@torch.no_grad()
def evaluate(model,loader,num_classes,device):
    model.eval(); ce=nn.CrossEntropyLoss()
    total_loss,total=0.0,0; conf=torch.zeros(num_classes,num_classes,dtype=torch.long,device=device)
    for x,lens,y in loader:
        x,lens,y=x.to(device),lens.to(device),y.to(device)
        logits=model(x,lens,train=False); loss=ce(logits,y)
        total_loss+=loss.item()*y.size(0); total+=y.size(0)
        pred=logits.argmax(dim=-1); idx=y*num_classes+pred
        binc=torch.bincount(idx,minlength=num_classes*num_classes)
        conf+=binc.view(num_classes,num_classes)
    tp=conf.diag().float(); fp=conf.sum(0).float()-tp; fn=conf.sum(1).float()-tp
    prec=torch.where(tp+fp>0,tp/(tp+fp),torch.zeros_like(tp))
    rec =torch.where(tp+fn>0,tp/(tp+fn),torch.zeros_like(tp))
    f1  =torch.where(prec+rec>0,2*prec*rec/(prec+rec),torch.zeros_like(tp))
    acc=(tp.sum()/conf.sum().clamp_min(1)).item()
    return (total_loss/max(total,1), acc, prec.mean().item(), rec.mean().item(), f1.mean().item())

def main():
    print("Device:", DEVICE, "| Mode: UNIdirectional")
    t0=time.time()
    tr=read_csv_flex(cfg.train); va=read_csv_flex(cfg.val); te=read_csv_flex(cfg.test)
    assert tr and va and te, "Pastikan train/val/test CSV ada & non-empty."
    vocab=build_vocab(tr,cfg.min_freq); labmap=build_labels(tr); C=len(labmap)
    print(f"Vocab={len(vocab)} | Labels={labmap}")
    ds_tr=TextDataset(tr,vocab,labmap,cfg.max_len); ds_va=TextDataset(va,vocab,labmap,cfg.max_len); ds_te=TextDataset(te,vocab,labmap,cfg.max_len)
    g=torch.Generator(); g.manual_seed(cfg.seed)
    dl_tr=DataLoader(ds_tr,batch_size=cfg.batch_size,shuffle=True,collate_fn=lambda b:collate_pad(b,0,cfg.max_len),generator=g)
    mkfix=lambda ds: DataLoader(ds,batch_size=cfg.batch_size,shuffle=False,collate_fn=lambda b:collate_pad(b,0,cfg.max_len))
    dl_va,dl_te=mkfix(ds_va),mkfix(ds_te)
    model=UniLstmClf(len(vocab),cfg.emb_dim,cfg.hidden,cfg.num_layers,C,cfg.dropout).to(DEVICE)
    opt=torch.optim.Adam(model.parameters(),lr=cfg.lr); ce=nn.CrossEntropyLoss()
    for ep in range(1,cfg.epochs+1):
        t_ep=time.time(); t_tr=time.time()
        model.train(); sum_loss=sum_acc=cnt=0
        for x,lens,y in dl_tr:
            x,lens,y=x.to(DEVICE),lens.to(DEVICE),y.to(DEVICE)
            logits=model(x,lens,train=True); loss=ce(logits,y)
            opt.zero_grad(set_to_none=True); loss.backward()
            if cfg.grad_clip: nn.utils.clip_grad_norm_(model.parameters(),cfg.grad_clip)
            opt.step()
            with torch.no_grad(): acc=(logits.argmax(-1)==y).float().mean().item()
            b=y.size(0); sum_loss+=loss.item()*b; sum_acc+=acc*b; cnt+=b
        tr_s=time.time()-t_tr
        v_loss,v_acc,p,r,f1=evaluate(model,dl_va,C,DEVICE); val_s=time.time()-t_ep-tr_s
        print(f"Epoch {ep:02d} | train loss {sum_loss/max(cnt,1):.4f} acc {sum_acc/max(cnt,1):.3f} | val loss {v_loss:.4f} acc {v_acc:.3f} P {p:.3f} R {r:.3f} F1 {f1:.3f} | time train {tr_s:.1f}s val {val_s:.1f}s total {time.time()-t_ep:.1f}s")
    t_test=time.time(); tl,ta,tp,trr,tf1=evaluate(model,dl_te,C,DEVICE)
    print(f"TEST | loss {tl:.4f} | acc {ta:.3f} | P {tp:.3f} R {trr:.3f} F1 {tf1:.3f} | time {time.time()-t_test:.1f}s")
    print(f"TOTAL {time.time()-t0:.1f}s")

if __name__=="__main__": main()


Device: cpu | Mode: UNIdirectional
Vocab=36148 | Labels={'negative': 0, 'positive': 1}
Epoch 01 | train loss 0.5086 acc 0.742 | val loss 0.3904 acc 0.825 P 0.826 R 0.825 F1 0.825 | time train 112.1s val 3.4s total 115.5s
Epoch 02 | train loss 0.3447 acc 0.848 | val loss 0.3503 acc 0.844 P 0.852 R 0.844 F1 0.844 | time train 109.4s val 3.4s total 112.9s
Epoch 03 | train loss 0.2774 acc 0.883 | val loss 0.3459 acc 0.853 P 0.854 R 0.853 F1 0.853 | time train 108.8s val 3.4s total 112.3s
Epoch 04 | train loss 0.2310 acc 0.907 | val loss 0.3236 acc 0.864 P 0.864 R 0.864 F1 0.864 | time train 109.1s val 3.5s total 112.5s
Epoch 05 | train loss 0.1919 acc 0.923 | val loss 0.3545 acc 0.866 P 0.867 R 0.866 F1 0.866 | time train 108.7s val 3.5s total 112.2s
Epoch 06 | train loss 0.1543 acc 0.940 | val loss 0.4248 acc 0.861 P 0.864 R 0.861 F1 0.860 | time train 108.8s val 3.3s total 112.2s
Epoch 07 | train loss 0.1267 acc 0.952 | val loss 0.4122 acc 0.866 P 0.867 R 0.866 F1 0.866 | time train 109.